In [1]:
# useing this variable for saving models and summaries
exp_name = "WCNNRNN_Wmultitask_16_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.WCNNRNN_model as WCNNRNN_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 7    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 2, 2]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [13]:
model = WCNNRNN_model.Model(n_variables = n_variables,
                            learning_rate = learning_rate,
                            n_classes = n_classes)

In [14]:
model.build_model(pos_weight = pos_weight, 
                 learning_rate = learning_rate,
                 wavelet_levels = wavelet_levels,    
                 convolution_levels = convolution_levels,    
                 pooling_stride = pooling_stride,     
                 weights_shape = weights_shape,
                 bias_shape = bias_shape,
                 wavelet_weights_shape = wavelet_weights_shape,    
                 activation = activation,
                 lstm_n_hidden = lstm_n_hidden,
                 trainable_wavelet = True,
                 trainable_conv = True,
                 trainable_last = True,
                 variables_weights = variables_weights
)


Tensor("Squeeze_24:0", shape=(?, ?, 19), dtype=float32) Tensor("Reshape_8:0", shape=(?, ?, 19), dtype=float32)


In [ ]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [ ]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.066 vs. 0.061
F1 = 0.097 vs. 0.090
Accuracy = 0.014 vs. 0.056
CHANGE DETECTION
AUC = 0.071 vs. 0.086
F1 = 0.155 vs. 0.158
Precision = 0.084 vs. 0.092
Recall = 0.994 vs. 0.550
Validation
ACTIVITY RECOGNITION
AUC = 0.060 vs. 0.058
F1 = 0.084 vs. 0.079
Accuracy = 0.009 vs. 0.056
CHANGE DETECTION
AUC = 0.047 vs. 0.051
F1 = 0.098 vs. 0.098
Precision = 0.051 vs. 0.054
Recall = 0.987 vs. 0.527
Minibatch Loss= 1.264979
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.650 vs. 0.063
F1 = 0.642 vs. 0.093
Accuracy = 0.598 vs. 0.056
CHANGE DETECTION
AUC = 0.222 vs. 0.098
F1 = 0.337 vs. 0.175
Precision = 0.229 vs. 0.096
Recall = 0.636 vs. 0.997
Validation
ACTIVITY RECOGNITION
AUC = 0.332 vs. 0.059
F1 = 0.395 vs. 0.082
Accuracy = 0.634 vs. 0.056
CHANGE DETECTION
AUC = 0.155 vs. 0.050
F1 = 0.244 vs. 0.102
Precision = 0.172 vs. 0.054
Recall = 0.421 vs. 0.932
Minibatch Loss= 0.377813
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.762 vs. 0.059


In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [25]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.631 vs. 0.057
F1 = 0.646 vs. 0.076
Accuracy = 0.867 vs. 0.056
CHANGE DETECTION
AUC = 0.144 vs. 0.027
F1 = 0.247 vs. 0.054
Precision = 0.176 vs. 0.028
Recall = 0.418 vs. 0.485


In [26]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [27]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [28]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [21]:
1

1

In [22]:
2

2